### Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
sys.path.append('../')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

### Configuration

In [3]:
from sv_system.utils.parser import set_train_config
import easydict

args = easydict.EasyDict(dict(dataset="voxc1_fbank_xvector", 
                              data_folder="/dataset/SV_sets/voxceleb12/feats/fbank64_vad/",
                              input_frames=400, splice_frames=[200, 400], stride_frames=1, 
                              input_format='fbank', input_dim=65, random_clip=True,
                              n_epochs=200, lrs=[0.1, 0.01], lr_schedule=[20], seed=1337,
                              no_eer=False, batch_size=64,
                              gpu_no=[0], cuda=True, num_workers=4,
                              arch="tdnn_conv", loss="softmax",
                             ))
config = set_train_config(args)

### Dataset

In [101]:
dev_df = pd.read_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_dev.csv")

In [113]:
held_out_spks = dev_df.spk.value_counts()[-10:].index
held_out_df = dev_df[dev_df.spk.isin(held_out_spks)]
held_out_train_df = held_out_df[held_out_df.set == 'train']
held_out_val_df = held_out_df[held_out_df.set == 'val']
held_out_train_dataset = FeatDataset.read_df(config, held_out_train_df, 'train')
held_out_val_dataset = FeatDataset.read_df(config, held_out_val_df, 'test')

In [103]:
dev_df = dev_df[~dev_df.spk.isin(held_out_spks)]
dev_train_df = dev_df[dev_df.set == 'train']
dev_val_df = dev_df[dev_df.set == 'val']
eval_df = pd.read_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_eval.csv") 

In [104]:
from sv_system.data.feat_dataset import FeatDataset

dev_train_dataset = FeatDataset.read_df(config, dev_train_df, 'train')
dev_val_dataset = FeatDataset.read_df(config, dev_val_df, 'test')
eval_dataset = FeatDataset.read_df(config, eval_df, 'test')

### Dataloader

In [105]:
from sv_system.data.dataloader import init_default_loader 
dev_train_dataloader = init_default_loader(config, dev_train_dataset, shuffle=True, var_len=False) 
dev_val_dataloader = init_default_loader(config, dev_val_dataset, shuffle=False, var_len=True) 
eval_dataloader = init_default_loader(config, eval_dataset, shuffle=False, var_len=True) 

In [114]:
held_out_train_dataloader = init_default_loader(config, held_out_train_dataset, shuffle=False, var_len=False)
held_out_val_dataloader = init_default_loader(config, held_out_val_dataset, shuffle=False, var_len=True)

### Model

In [129]:
from tdnn_models import tdnn_xvector

### Fintune Model

In [130]:
import torch

model = tdnn_xvector(config, 512, n_labels=len(dev_df.label.unique())+10)
# saved_model = torch.load("trained_models/vox1_dev_tdnn_xvector_held_out.pt")
# model.load_state_dict(saved_model)

if not config['no_cuda']:
    model = model.cuda()

In [ ]:
import torch.nn as nn
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, MultiStepLR
from sklearn.metrics import roc_curve
import torch.nn.functional as F
from tensorboardX import SummaryWriter

optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)
# criterion = nn.CrossEntropyLoss(weight=label_weights)
criterion = nn.CrossEntropyLoss()
plateau_scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5)
# step_scheduler = MultiStepLR(optimizer, [30], 0.1)

writer = SummaryWriter("logs/held_out")
model_path = "trained_models/vox1_dev_tdnn_xvector_held_out.pt"
start_epoch = 0

for epoch_idx in range(start_epoch, config['n_epochs']):
    print("-"*30)
    curr_lr = optimizer.state_dict()['param_groups'][0]['lr']
    print("curr_lr: {}".format(curr_lr))
    
# =============== train code #===============
    model.train()
    loss_sum = 0
    n_corrects = 0
    total = 0
    for batch_idx, (X, y) in enumerate(dev_train_dataloader):
        if not config['no_cuda']:
            X = X.cuda()
            y = y.cuda()

        optimizer.zero_grad()
        logit = model(X)
        loss = criterion(logit, y)
        loss.backward()
        optimizer.step()
                        
        loss_sum += loss.item()
        n_corrects += logit.max(1)[1].eq(y).sum().item()
        total += y.size(0)
#         if (batch_idx+1) % 1000 == 0:
#             print("Batch {}/{}\t Loss {:.6f}" \
#                   .format(batch_idx+1, len(si_loader), loss_sum / total))
    train_loss = loss_sum / total
    train_acc = n_corrects / total
    plateau_scheduler.step(train_loss)
    
    print("epoch #{}, train loss: {:.4f}, train acc: {:.4f}".format(epoch_idx, train_loss, train_acc))
    writer.add_scalar("train/loss", train_loss, epoch_idx+1)
    writer.add_scalar("train/acc", train_acc, epoch_idx+1)

#=============== dev_val code #===============
    model.eval()
    loss_sum = 0
    n_corrects = 0
    total = 0
    for batch_idx, (X, y) in enumerate(dev_val_dataloader):
        if not config['no_cuda']:
            X = X.cuda()
            y = y.cuda()

        logit = model(X)
        loss = criterion(logit, y)
        loss_sum += loss.item()
        n_corrects += logit.max(1)[1].eq(y).sum().item()
        total += y.size(0)
    val_loss = loss_sum / total
    val_acc = n_corrects / total
    
    print("epoch #{}, val loss: {:.4f}, val acc: {:.4f}".format(epoch_idx, val_loss, val_acc))
    writer.add_scalar("val/loss", val_loss, epoch_idx+1)
    writer.add_scalar("val/acc", val_acc, epoch_idx+1)
    torch.save(model.state_dict(), model_path)

In [127]:
# See, Fr features
fr_feats = []
model.eval()
total = 0
for batch_idx, (X, y) in enumerate(dev_val_dataloader):
    if not config['no_cuda']:
        X = X.cuda()
        y = y.cuda()

    fr_feat = model.fr_feat(X).cpu().detach()
    fr_feats.append(fr_feat)
    break

In [128]:
from torch.nn.functional import cosine_similarity as cosine
cosine(fr_feats[0][0,:,0:1], fr_feats[0][0,:,1:], dim=0)

tensor([0.9226, 0.7637, 0.5905, 0.4650, 0.3965, 0.3624, 0.3237, 0.2873, 0.2718,
        0.2553, 0.2583, 0.2505, 0.2527, 0.2539, 0.2557, 0.2563, 0.2401, 0.2268,
        0.2261, 0.2295, 0.2399, 0.2526, 0.2675, 0.2725, 0.2617, 0.2381, 0.2301,
        0.2367, 0.2417, 0.2536, 0.2603, 0.2381, 0.2207, 0.1969, 0.1837, 0.1872,
        0.1942, 0.2218, 0.2566, 0.2421, 0.2378, 0.2361, 0.2238, 0.2398, 0.2526,
        0.2569, 0.2780, 0.2980, 0.2748, 0.2692, 0.2281, 0.1951, 0.1903, 0.1980,
        0.2044, 0.2165, 0.2160, 0.1929, 0.1900, 0.2037, 0.2059, 0.1945, 0.2040,
        0.2211, 0.2484, 0.2909, 0.3278, 0.3465, 0.3434, 0.3335, 0.3265, 0.3066,
        0.2900, 0.2866, 0.3043, 0.3135, 0.3199, 0.3052, 0.2881, 0.2666, 0.2464,
        0.2296, 0.2084, 0.1941, 0.1960, 0.1970, 0.2088, 0.2268, 0.2424, 0.2605,
        0.2513, 0.2701, 0.2541, 0.2128, 0.1948, 0.2012, 0.2227, 0.2383, 0.2599,
        0.2907, 0.3295, 0.3709, 0.4564, 0.4619, 0.3714, 0.3026, 0.2717, 0.2558,
        0.2204, 0.2095, 0.1857, 0.1725, 

###  Freeze Model & FineTurne

In [117]:
freezed_params = []
for param_name, param in model.named_parameters():
    if param_name not in ['classifier.2.weight', 'classifier.2.bias']:
        freezed_params.append(param)
        
for param in freezed_params:
    param.requires_grad = False

In [121]:
# =============== fine_tune code #===============
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)
for epoch_idx in range(0, 10):
    print("-"*30)
    curr_lr = optimizer.state_dict()['param_groups'][0]['lr']
    print("curr_lr: {}".format(curr_lr))
    model.train()
    loss_sum = 0
    n_corrects = 0
    total = 0
    for batch_idx, (X, y) in enumerate(held_out_train_dataloader):
        if not config['no_cuda']:
            X = X.cuda()
            y = y.cuda()

        optimizer.zero_grad()
        logit = model(X)
        loss = criterion(logit, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        n_corrects += logit.max(1)[1].eq(y).sum().item()
        total += y.size(0)
    #         if (batch_idx+1) % 1000 == 0:
    #             print("Batch {}/{}\t Loss {:.6f}" \
    #                   .format(batch_idx+1, len(si_loader), loss_sum / total))
    train_loss = loss_sum / total
    train_acc = n_corrects / total

    print("epoch #{}, train loss: {:.4f}, train acc: {:.4f}".format(epoch_idx, train_loss, train_acc))

------------------------------
curr_lr: 0.01
epoch #0, train loss: 0.0219, train acc: 0.9818
------------------------------
curr_lr: 0.01
epoch #1, train loss: 0.0217, train acc: 0.9818
------------------------------
curr_lr: 0.01
epoch #2, train loss: 0.0214, train acc: 0.9848
------------------------------
curr_lr: 0.01
epoch #3, train loss: 0.0210, train acc: 0.9848
------------------------------
curr_lr: 0.01
epoch #4, train loss: 0.0206, train acc: 0.9848
------------------------------
curr_lr: 0.01
epoch #5, train loss: 0.0202, train acc: 0.9848
------------------------------
curr_lr: 0.01
epoch #6, train loss: 0.0198, train acc: 0.9848
------------------------------
curr_lr: 0.01
epoch #7, train loss: 0.0194, train acc: 0.9848
------------------------------
curr_lr: 0.01
epoch #8, train loss: 0.0190, train acc: 0.9848
------------------------------
curr_lr: 0.01
epoch #9, train loss: 0.0186, train acc: 0.9848


In [123]:
#=============== dev_val code #===============
model.eval()
loss_sum = 0
n_corrects = 0
total = 0
for batch_idx, (X, y) in enumerate(held_out_val_dataloader):
    if not config['no_cuda']:
        X = X.cuda()
        y = y.cuda()

    logit = model(X)
    loss = criterion(logit, y)
    loss_sum += loss.item()
    n_corrects += logit.max(1)[1].eq(y).sum().item()
    total += y.size(0)
val_loss = loss_sum / total
val_acc = n_corrects / total
print("epoch #{}, val loss: {:.4f}, val acc: {:.4f}".format(epoch_idx, val_loss, val_acc))

epoch #9, val loss: 0.0389, val acc: 0.8519


### SV Test

### LDA on embedding

In [20]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

global_mean = si_embeds.mean(0)
clf = LDA(solver='svd', n_components=200)
clf.fit(si_embeds - global_mean, si_key_df.label)

si_embeds = clf.transform(si_embeds - global_mean).astype(np.float32)

sv_embeds = clf.transform(sv_embeds - global_mean).astype(np.float32)

si_dataset, embed_dim, n_labels = embedToDataset(si_embeds.reshape(-1,200), si_key_df)
sv_dataset, _, _ = embedToDataset(sv_embeds, sv_key_df)